Recurrent transaction matcher:  users features from previous and current transaction

Looking at the dat in random forest training : binary classification True or False



https://github.com/meetcleo/data-science-services/blob/master/recurring_transaction_matcher/scripts/train_similarity_model/random_forest_training.py

In [21]:
import pandas as pd
from datetime import datetime
import boto3
from botocore.exceptions import ClientError
from io import StringIO
from cleodata.utils.secrets import get_secret
import s3fs
from fastparquet import ParquetFile
from cleodata.sources.sync.sync import SyncDataSource
boto3.setup_default_session(profile_name='DataScientist-878877078763')
redshift_source = SyncDataSource("data_exploration", use_redshift=True, redshift_cluster="cleo-production-redshift", redshift_db="cleo")

2024-05-29 14:07:32 [debug    ] fetching credentials          
2024-05-29 14:07:34 [info     ] Credentials acquired          
2024-05-29 14:07:34 [info     ] Built connection pool         


In [22]:
def read_from_s3(path):
    """Read parquet files and combine them into a single dataframe"""
    fs = s3fs.core.S3FileSystem()
    all_paths_from_s3 = fs.glob(path=f"{path}*.parquet")

    if len(all_paths_from_s3) > 0:
        s3 = s3fs.S3FileSystem()
        fp_obj = ParquetFile(
            all_paths_from_s3, open_with=s3.open
        )  # use s3fs as the filesystem
        data = fp_obj.to_pandas()
        return data
    elif len(all_paths_from_s3)==1:
        return pd.read_parquet(all_paths_from_s3[0])
    else:
        print(f"Nothing found")
        print(f"paths from a{all_paths_from_s3}")
    
def read_csv_s3(bucket, key):
    try:
        s3 = boto3.client('s3')
        obj = s3.get_object(Bucket=bucket, Key=key)
        df = pd.read_csv(obj['Body'])
        return df
    except ClientError as ex:
        if ex.response['Error']['Code'] == 'NoSuchKey':
            print("Key doesn't match. Please check the key value entered.")


def list_s3_flies(base_path):
    fs = s3fs.core.S3FileSystem()
    all_paths_from_s3 = fs.glob(path=f"{base_path}*.parquet")
    return all_paths_from_s3


In [3]:
path_data = "s3://cleo-data-science/recurring_transaction_matcher/training_data/data_2021_11_11__13_56_14.csv"

In [6]:
bucket = "cleo-data-science"
key = "recurring_transaction_matcher/training_data/data_2021_11_11__13_56_14.csv"
df_data = read_csv_s3(bucket, key)

In [7]:
df_data

,is_match,amount_distance,cycle_offset_distance,cycles_delayed_distance,date_distance,historical_date_offset,is_bill,is_not_prefix,is_same_account,merchant_edit_distance,merchant_is_not_prefix,merchant_weighted_token_distance,past_edit_distance,past_is_not_prefix,past_weighted_token_distance,transaction_id,recurring_transaction_id
0,False,0.074222,0.066667,0.022840,0.044753,0.000000,1.0,1.0,1.0,0.906250,1.0,1.0,0.906250,1.0,1.0,3285381036,5826732
1,True,0.162866,0.466667,0.149333,0.308000,0.166667,0.0,1.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,3373322101,ecf1a78c-7539-4887-90f8-89256e24dff5
2,False,0.889039,0.133333,0.045158,0.089246,0.000000,0.0,1.0,1.0,0.960000,1.0,1.0,0.827586,1.0,1.0,3453444374,04f6eb75-686f-4055-8373-180f50b58c28
3,False,0.249575,0.266667,0.088278,0.177472,0.000000,1.0,1.0,1.0,0.700000,1.0,1.0,0.869565,1.0,1.0,3381116159,6458340
4,False,0.987022,0.066667,0.022840,0.044753,0.000000,0.0,1.0,1.0,0.904762,1.0,1.0,0.842105,1.0,1.0,3240660860,257533b9-0059-48a8-979a-3716f29132f4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76169,False,0.927397,1.000000,0.292893,0.646447,0.000000,0.0,1.0,1.0,0.680851,1.0,1.0,0.711538,1.0,1.0,3254873236,18fca700-3d26-4d21-a6a0-679c21bb4aea
76170,False,0.238095,0.200000,0.066967,0.133484,0.000000,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,3457522971,6477229
76171,False,0.243000,0.200000,0.066967,0.133484,0.000000,1.0,1.0,1.0,0.857143,1.0,1.0,0.857143,1.0,1.0,3230182385,6263069
76172,False,0.214907,0.400000,0.129449,0.264725,0.000000,1.0,1.0,1.0,0.888889,1.0,1.0,0.924528,1.0,1.0,3475457080,6557422


In [8]:
df_data.columns

Index(['is_match', 'amount_distance', 'cycle_offset_distance',
       'cycles_delayed_distance', 'date_distance', 'historical_date_offset',
       'is_bill', 'is_not_prefix', 'is_same_account', 'merchant_edit_distance',
       'merchant_is_not_prefix', 'merchant_weighted_token_distance',
       'past_edit_distance', 'past_is_not_prefix',
       'past_weighted_token_distance', 'transaction_id',
       'recurring_transaction_id'],
      dtype='object')

In [11]:
df_data[['is_match','transaction_id','recurring_transaction_id']][0:5]

,is_match,transaction_id,recurring_transaction_id
0,False,3285381036,5826732
1,True,3373322101,ecf1a78c-7539-4887-90f8-89256e24dff5
2,False,3453444374,04f6eb75-686f-4055-8373-180f50b58c28
3,False,3381116159,6458340
4,False,3240660860,257533b9-0059-48a8-979a-3716f29132f4


In [13]:
df_data['is_match'].value_counts(dropna=False)

is_match
False    60462
True     15712
Name: count, dtype: int64

In [20]:
df_data.groupby('is_match').agg({'amount_distance':'mean',\
                                 'is_same_account': 'mean',\
                                    'merchant_is_not_prefix':'mean',\
                                       'merchant_weighted_token_distance':'mean',\
                                          'date_distance':'mean',\
                                             'cycles_delayed_distance':'mean'})

,amount_distance,is_same_account,merchant_is_not_prefix,merchant_weighted_token_distance,date_distance,cycles_delayed_distance
is_match,,,,,,
False,0.475871,0.886358,0.961695,0.961024,0.277106,0.213452
True,0.228107,0.943292,0.399122,0.413670,0.316643,0.294245


Date distance and cycles_delayed_distance are higher in matches? Date distnace to what?


https://www.notion.so/meetcleo/Similarity-based-matching-e75bfc7efd684cefb541787e7d05ac23#014f1d707c4b411793f5823adad46763
